In [7]:
from collections import deque
import time


class Order:
    def __init__(self, order_id, side, qty, price=None, order_type="LIMIT"):
        self.id = order_id
        self.side = side              # "BUY" or "SELL"
        self.qty = qty
        self.price = price
        self.type = order_type        # "LIMIT" or "MARKET"
        self.timestamp = time.time()  # used for FIFO


class OrderBook:
    def __init__(self):
        self.bids = {}  # price -> deque([Order,...])
        self.asks = {}
        self.trade_log = []

    # ---------- Utilities ----------
    def best_bid(self):
        return max(self.bids.keys()) if self.bids else None

    def best_ask(self):
        return min(self.asks.keys()) if self.asks else None

    def add_limit_order(self, order):
        book = self.bids if order.side == "BUY" else self.asks
        if order.price not in book:
            book[order.price] = deque()
        book[order.price].append(order)

    def record_trade(self, price, qty, buy_id, sell_id):
        self.trade_log.append({
            "price": price,
            "qty": qty,
            "buyer_id": buy_id,
            "seller_id": sell_id,
            "timestamp": time.time()
        })

    # ---------- Matching Logic ----------
    def match(self, incoming):
        """
        Core price-time priority matching engine.
        Walks the book until no more match is possible or qty=0.
        """

        def executable():
            if incoming.type == "MARKET":
                # Market orders ignore price, match best available
                return self.best_ask() if incoming.side == "BUY" else self.best_bid()
            else:
                if incoming.side == "BUY":
                    return self.best_ask() is not None and incoming.price >= self.best_ask()
                else:
                    return self.best_bid() is not None and incoming.price <= self.best_bid()

        while incoming.qty > 0 and executable():

            if incoming.side == "BUY":
                price = self.best_ask()
                level = self.asks[price]
                resting = level[0]
            else:
                price = self.best_bid()
                level = self.bids[price]
                resting = level[0]

            executed = min(incoming.qty, resting.qty)

            # reduce quantities
            incoming.qty -= executed
            resting.qty -= executed

            # record trade
            if incoming.side == "BUY":
                self.record_trade(price, executed, incoming.id, resting.id)
            else:
                self.record_trade(price, executed, resting.id, incoming.id)

            # remove exhausted resting order
            if resting.qty == 0:
                level.popleft()
                if not level:
                    if incoming.side == "BUY":
                        del self.asks[price]
                    else:
                        del self.bids[price]

        # if it’s a limit order with remaining qty → add to book
        if incoming.type == "LIMIT" and incoming.qty > 0:
            self.add_limit_order(incoming)

    # Wrapper APIs
    def limit_buy(self, order_id, qty, price):
        self.match(Order(order_id, "BUY", qty, price, "LIMIT"))

    def limit_sell(self, order_id, qty, price):
        self.match(Order(order_id, "SELL", qty, price, "LIMIT"))

    def market_buy(self, order_id, qty):
        self.match(Order(order_id, "BUY", qty, None, "MARKET"))

    def market_sell(self, order_id, qty):
        self.match(Order(order_id, "SELL", qty, None, "MARKET"))

    # Debug helpers
    def print_book(self):
        print("\nASKS (lowest → highest)")
        for p in sorted(self.asks.keys()):
            total = sum(o.qty for o in self.asks[p])
            print(f"{p}: {total}")

        print("\nBIDS (highest → lowest)")
        for p in sorted(self.bids.keys(), reverse=True):
            total = sum(o.qty for o in self.bids[p])
            print(f"{p}: {total}")

        print()


# ==========================================================
#  VALIDATION TEST (MUST PASS)
# ==========================================================
if __name__ == "__main__":
    ob = OrderBook()

    # Submit ASK ladder
    ob.limit_sell("S1", 10, 101)
    ob.limit_sell("S2", 20, 102)
    ob.limit_sell("S3", 30, 103)

    print("Before MARKET BUY:")
    ob.print_book()

    # Massive MARKET BUY for qty=60
    ob.market_buy("B1", 60)

    print("After MARKET BUY:")
    ob.print_book()

    print("Trade Log:")
    for t in ob.trade_log:
        print(t)


Before MARKET BUY:

ASKS (lowest → highest)
101: 10
102: 20
103: 30

BIDS (highest → lowest)

After MARKET BUY:

ASKS (lowest → highest)

BIDS (highest → lowest)

Trade Log:
{'price': 101, 'qty': 10, 'buyer_id': 'B1', 'seller_id': 'S1', 'timestamp': 1767678074.0476468}
{'price': 102, 'qty': 20, 'buyer_id': 'B1', 'seller_id': 'S2', 'timestamp': 1767678074.0476563}
{'price': 103, 'qty': 30, 'buyer_id': 'B1', 'seller_id': 'S3', 'timestamp': 1767678074.0476606}
